# 감정선 classification

감정선

In [17]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [20]:
df = pd.read_excel('label4.xlsx')
df = df[~df['label'].str.contains("-")]  # Filter out invalid entries

In [21]:
print(df)

     File Number  life head  heart label
0              1     0    0      0   000
1              2     0    0      0   000
2              5     0    1      0   010
3             12     0    0      1   001
4             13     0    1      0   010
..           ...   ...  ...    ...   ...
976         5353     0    0      1   001
977         5354     1    0      1   101
978         5366     0    0      0   000
979         5367     0    0      0   000
981         5385     0    0      1   001

[927 rows x 5 columns]


In [22]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [23]:
def load_and_preprocess_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./head', f"image{row['File Number']}.png")
        image = Image.open(image_path).resize((128, 128))
        image = np.array(image) / 255.0
        if image.ndim == 2 or image.shape[2] == 1:
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB
        images.append(image)
    return np.array(images)

In [24]:
train_images = load_and_preprocess_images(train_df)
test_images = load_and_preprocess_images(test_df)

  0%|          | 0/741 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

In [25]:
train_labels = np.array(train_df['head']).astype('float32')
test_labels = np.array(test_df['head']).astype('float32')

In [26]:
print(train_df['head'].value_counts())

0    528
1    213
Name: head, dtype: int64


In [27]:
# Initialize MobileNetV2 model with pretrained ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [28]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [29]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True)
]

history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # May increase if model is not overfitting
    callbacks=callbacks
)

# Load the best weights and unfreeze the base_model for fine-tuning
model.load_weights('best_model.h5')
base_model.trainable = True

Epoch 1/50
24/24 [==============================] - 17s 524ms/step - loss: 0.6943 - binary_accuracy: 0.6748 - val_loss: 0.6229 - val_binary_accuracy: 0.6882
Epoch 2/50
24/24 [==============================] - 11s 478ms/step - loss: 0.5539 - binary_accuracy: 0.7503 - val_loss: 0.6065 - val_binary_accuracy: 0.6720
Epoch 3/50
24/24 [==============================] - 9s 373ms/step - loss: 0.5599 - binary_accuracy: 0.7544 - val_loss: 0.6195 - val_binary_accuracy: 0.6720
Epoch 4/50
24/24 [==============================] - 12s 511ms/step - loss: 0.5237 - binary_accuracy: 0.7638 - val_loss: 0.6314 - val_binary_accuracy: 0.6774
Epoch 5/50
24/24 [==============================] - 9s 376ms/step - loss: 0.5342 - binary_accuracy: 0.7692 - val_loss: 0.6431 - val_binary_accuracy: 0.6720
Epoch 6/50
24/24 [==============================] - 10s 409ms/step - loss: 0.5087 - binary_accuracy: 0.7706 - val_loss: 0.6194 - val_binary_accuracy: 0.6667
Epoch 7/50
24/24 [==============================] - 10s 405m

In [30]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 2s 256ms/step - loss: 0.6065 - binary_accuracy: 0.6720
Test accuracy: 67.20%


In [31]:
from sklearn.metrics import f1_score

# Predict on the test set
test_predictions = model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Calculate the F1 score
f1 = f1_score(test_labels, test_predictions)

print(f"F1 Score: {f1}")

6/6 [==============================] - 3s 391ms/step
F1 Score: 0.2823529411764706


In [32]:
# Recompile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [33]:
# Fine-tune the model
history_fine = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
24/24 [==============================] - 66s 2s/step - loss: 1.0981 - binary_accuracy: 0.3212 - val_loss: 0.6155 - val_binary_accuracy: 0.6882
Epoch 2/20
24/24 [==============================] - 47s 2s/step - loss: 0.9078 - binary_accuracy: 0.4035 - val_loss: 0.6358 - val_binary_accuracy: 0.6882
Epoch 3/20
24/24 [==============================] - 48s 2s/step - loss: 0.8021 - binary_accuracy: 0.4723 - val_loss: 0.6622 - val_binary_accuracy: 0.6774
Epoch 4/20
24/24 [==============================] - 48s 2s/step - loss: 0.6931 - binary_accuracy: 0.5722 - val_loss: 0.6823 - val_binary_accuracy: 0.6774
Epoch 5/20
24/24 [==============================] - 44s 2s/step - loss: 0.6355 - binary_accuracy: 0.6424 - val_loss: 0.6808 - val_binary_accuracy: 0.6882
Epoch 6/20
24/24 [==============================] - 48s 2s/step - loss: 0.5810 - binary_accuracy: 0.6977 - val_loss: 0.6649 - val_binary_accuracy: 0.6882
Epoch 6: early stopping


In [34]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 2s 252ms/step - loss: 0.6649 - binary_accuracy: 0.6882
Test accuracy: 68.82%


In [35]:
from sklearn.metrics import f1_score

# Predict on the test set
test_predictions = model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Calculate the F1 score
f1 = f1_score(test_labels, test_predictions)

print(f"F1 Score: {f1}")

6/6 [==============================] - 2s 249ms/step
F1 Score: 0.2368421052631579
